# Construction of the pLBL2 vector

The pLBL2 vector is a yeast/E. coli shuttle vector for expression of genes in S. cerevisiae.
It is a derivative of pYPK7_TEF1_RPS7B but with a URA3 marker instead of the kanMX4 of pYPK7_TEF1_RPS7B.

An AjiI site was removed from the 2µ sequence during the construction process.

In [1]:
from pydna.all import *

### Larger fragment from pYPK7_TEF1_RPS7B

This fragment contains The TEF1 promoter and RPS7B terminator, amp marker and a part of the 2µ ORI

In [2]:
pYPK7_TEF1_RPS7B = read("pYPK7_TEF1_RPS7B.gb")

In [3]:
f4008 = pYPK7_TEF1_RPS7B[5399:1866]

In [4]:
f4008

Dseqrecord(-4008)

### Smaller fragment from pYPKpw

the 2.9kb fragment contains the pUC origin, URA3 and a part of the 2µ

In [5]:
pYPKpw = read("pYPKpw.gb")

In [6]:
f2955 = pYPKpw[505:3460]

In [7]:
f2955.seq

Dseq(-2955)
CTTC..GCAC
GAAG..CGTG

The theoretically expected sequence of pLBL2 is the two fragments (f4008, f2955) ligated together with one mutated nucleotide 
to form a 6964 bp circular DNA molecule. The nucleotide can be A or C or T (IUPAC DNA code H ) in order to remove one AjiI site from the 2µ sequence. 

In [8]:
theoretical_pLBL2_sequence = (f2955+Dseqrecord("h")+f4008).looped().synced(pYPKpw)
theoretical_pLBL2_sequence, theoretical_pLBL2_sequence.cseguid()

(Dseqrecord(o6964), Y9N9GHq6Rkv4mVbKbZXcWmMSOg0)

Design two primer pairs for the gibson assembly of pLBL2. 
The objects below are instances of the Amplicon class storing a forward primers and a reverse primer.

In [9]:
f2955product = primer_design(f2955)
f4008product = primer_design(f4008)

The f2955tails, f4008tails are modified with tails to allow assembly by homologous recombination. The "h" nucleotide is incorporated into the 
PCR primer tails.

In [10]:
f2955tails, f4008tails = circular_assembly_fragments((f2955product, Dseqrecord("h"),f4008product), overlap=25)

In [11]:
f2955tails, f4008tails

(Amplicon(2982), Amplicon(4035))

The two fragments are assembled by homologous recombination.

In [12]:
f2955tails.name="f2955"
f4008tails.name="f4008"

In [13]:
asm = Assembly((f2955tails, f4008tails), limit=25)
asm

Assembly
fragments..: 2982bp 4035bp
limit(bp)..: 25
G.nodes....: 6
algorithm..: common_sub_strings

In [14]:
candidates = asm.assemble_circular()
candidates

[Contig(o6964),
 Contig(o6964),
 Contig(o4871),
 Contig(o4871),
 Contig(o2093),
 Contig(o2093)]

Apparently several circular products are possible. Care should be taken when confirming this construct.
We assume that the first one is correct since it has the expected size.

In [15]:
candidate = candidates[0]
candidate

Contig(o6964)

In [16]:
pYPK0_TEF1_RPS7B = candidate.synced(pYPKpw)
pYPK0_TEF1_RPS7B.cseguid()

Y9N9GHq6Rkv4mVbKbZXcWmMSOg0

In [17]:
pLBL2 = pYPK0_TEF1_RPS7B
assert pLBL2.seguid() == theoretical_pLBL2_sequence.seguid() == 'ouH9liSpWQvjZEhoT3kScn2aTHk'

In [18]:
print(f2955tails.forward_primer.format("tab"))
print(f2955tails.reverse_primer.format("tab"))
print(f4008tails.forward_primer.format("tab"))
print(f4008tails.reverse_primer.format("tab"))

f2955	GAAATTTCAGCCACTTCACAGGCGGTT

r2955	ACCGGGGATGCGAdGTGCAAGATTACCTATGC

f4008	GGTAATCTTGCAChTCGCATCCCCGGT

r4008	ACCGCCTGTGAAGTGGCTGAAATTTCGTACAAAAAATTTC



The primers below were bought from IDT. They were designed manually, so they are not identical to the ones above.

In [19]:
p1s,p2s,p3s,p4s = parse_primers('''
>2954fw
GAA ATT TCA GCC ACT TCA CAG GCG GTT TT
>2954rv2
CCGGGGATGCGAdgTGCAAGATTACCTATGC
>4009fw2
AGGTAATCTTGCAchTCGCATCCCCGGTTC
>4009rv
ACC GCC TGT GAA GTG GCT GAA ATT TCG TA''')

To confirm that the primers used results in the same result, the simulation above is repeated with the new primers.

In [20]:
product_2954fw_2954rv2 = pcr(p1s, p2s, pYPKpw)

In [21]:
product_4009fw2_4009rv = pcr(p3s,p4s,pYPK7_TEF1_RPS7B)

In [22]:
asm = Assembly((product_2954fw_2954rv2, product_4009fw2_4009rv), limit=25)

In [23]:
candidate = candidates[0]

In [24]:
assert pLBL2.cseguid() == candidate.cseguid() == 'Y9N9GHq6Rkv4mVbKbZXcWmMSOg0'

The plasmid assembled with the purchased primers is identical to the planned as can be seen form their identical cseguid checksums.

pLBL2 should only have one AjiI site, as this was one of the design goals.

In [25]:
from Bio.Restriction import AjiI

In [26]:
pLBL2.linearize(AjiI)

Dseqrecord(-6964)

In [27]:
pLBL2.locus = "pLBL2"
pLBL2.definition = "pYPK0_TEF1_RPS7B"

In [28]:
pLBL2.stamp()

cSEGUID_Y9N9GHq6Rkv4mVbKbZXcWmMSOg0

In [29]:
pLBL2.write("pLBL2.gb")

pLBL2.gb

### Analysis

Two colony PCRs were performed to verify the plasmid. One amplifies the ura3 marker and the other the 2µ sequence.

In [30]:
URA3f, URA3r, _2mu_f, _2mu_r = parse('''

>17_URA3f
TAA TAA CTG ATA TAA TTA AAT TGA AGC

>18_URA3r
CGG TTT CTT TGA AAT TTT TTT GAT TCG

>19_2mu_f
GCG CTT TAC GGA AGA CAA TGT

>20_2mu_r
GGG AAG TGG AGT CAG GCT TT

''', ds=False)

In [31]:
pcr(URA3f, URA3r, pLBL2)

Amplicon(1056)

In [32]:
pcr(_2mu_f, _2mu_r, pLBL2)

Amplicon(602)